In [1]:
# Requisitos
from gensim.models import TfidfModel  # obliga a TfidfModelo mitgana normal
from gensim.utils import simple_preprocess #tokanitzar i min
from gensim.corpora import Dictionary
from gensim.models import Word2Vec
import numpy as np

In [2]:
# Tipado
from typing import Tuple, List


# Part 1:  Entrenament de models

## Word2vec

### Dades

In [ ]:
from datasets import load_dataset
dataset = load_dataset("projecte-aina/catalan_general_crawling")

In [ ]:
train_data = dataset["train"]

with open("datos.txt", "w", encoding="utf-8") as file:
    for example in train_data:
        text = example["text"]
        file.write(text + "\n")


KeyboardInterrupt: 

In [ ]:
!head -c 100M datos.txt > subset_100.txt


In [ ]:
!head -c 500M datos.txt > subset_500.txt

### Entrenament

In [36]:
from gensim.models import word2vec

In [47]:
# Cargar el archivo de texto
sentences = word2vec.LineSentence("subset100.txt")
model = word2vec.Word2Vec(sentences, vector_size=100, window=5, min_count=10, workers=4, epochs=25)
model.save('models/word2vec_model_100.bin')

In [52]:
sentences2 = word2vec.LineSentence("subset_500.txt")
model2 = word2vec.Word2Vec(sentences2, vector_size=100, window=5, min_count=10, workers=4, epochs=25)
model2.save('models/word2vec_model_500.bin')

In [48]:
model100 = Word2Vec.load('models/word2vec_model_100.bin')

In [53]:
model500 = Word2Vec.load('models/word2vec_model_500.bin')

## Fasttext

### Carrega del model

In [ ]:
from gensim.models import fasttext
# Modelos pre-entrenados
WV_MODEL_PATH = "models/cc.ca.300.bin.gz" #model fast text 
wv_model = fasttext.load_facebook_vectors(WV_MODEL_PATH)
# Heu de guardar el model en un format compatible  #1 pas memory map
wv_model.save('cc.ca.gensim.bin')


In [265]:
# Llavors podeu carregar el model com a mmap #2 pas memory map
from gensim.models.fasttext import FastTextKeyedVectors
ft_model = FastTextKeyedVectors.load('models/cc.ca.gensim_loaded.bin', mmap='r')


UnpicklingError: unpickling stack underflow

## Resultats
falta provar per model100 vs model500 (word2vec) vs ft_model(fasttext) 

In [ ]:
model100.wv['parlament'] #ja tenim una representació da cada paraula en forma de vector

In [ ]:
ft_model.doesnt_match(["cadira", "sofa", "gat", "butaca"])

In [ ]:
model500.wv.most_similar("parlament", topn=5)

In [ ]:
model100.wv.most_similar("parlament", topn=5)

In [ ]:
model500.wv.most_similar(positive=["banc", "cadira"], negative=["diners"], topn=5)


In [ ]:
model500.wv.doesnt_match(["cadira", "sofa", "gat", "butaca"])

# Part 2: Semantic Text Similarity
Este modelo utiliza gensim para convertir pares de vectores + puntuaciones en vectores (word embeddings).
Dado un dataset, infiere la puntuación de similitud entre ambas frases.

In [3]:
#importar llibreries
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import OneHotEncoder

In [4]:
#carregar dades
from datasets import load_dataset
dataset = load_dataset("projecte-aina/sts-ca")  

Found cached dataset sts-ca (C:/Users/llumf/.cache/huggingface/datasets/projecte-aina___sts-ca/StsCa/1.0.2/bad37fb7fb0f06f3d2316e29637293b25160a93a24f36f1974f21313ac2f3342)


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
modeloss = []
P_Embeding = []
P_Model = []

In [5]:
train = dataset['train'] #datasets.dataset_dict.DatasetDict
train = train[0:len(train)] # canvia format de dataset a diccionari

validation = dataset['validation']
validation = validation[0:len(validation)] # canvia format de dataset a diccionari

test = dataset['test']
test = test[0:len(test)] # canvia format de dataset a diccionari

In [6]:
#convertir a Dataframe
df_tr = pd.DataFrame(train)
df_val = pd.DataFrame(validation)
df_ts = pd.DataFrame(test)

In [7]:
#Convertir a Llista
input_pairs = df_tr.values.tolist()
val = df_val.values.tolist()
ts = df_ts.values.tolist()

dades = input_pairs + val + ts

## Funcions comunes

### Preprocessing

In [8]:
# Cargar stopwords en Catalan
#STOPWORDS_CA = {"a", "abans", "ací", "ah", "així", "això", "al", "aleshores", "algun", "alguna", "algunes", "alguns", "alhora", "allà", "allí", "allò", "als", "altra", "altre", "altres", "amb", "ambdues", "ambdós", "anar", "ans", "apa", "aquell", "aquella", "aquelles", "aquells", "aquest", "aquesta", "aquestes", "aquests", "aquí", "baix", "bastant", "bé", "cada", "cadascuna", "cadascunes", "cadascuns", "cadascú", "com", "consegueixo", "conseguim", "conseguir", "consigueix", "consigueixen", "consigueixes", "contra", "d'un", "d'una", "d'unes", "d'uns", "dalt", "de", "del", "dels", "des", "des de", "després", "dins", "dintre", "donat", "doncs", "durant", "e", "eh", "el", "elles", "ells", "els", "em", "en", "encara", "ens", "entre", "era", "erem", "eren", "eres", "es", "esta", "estan", "estat", "estava", "estaven", "estem", "esteu", "estic", "està", "estàvem", "estàveu", "et", "etc", "ets", "fa", "faig", "fan", "fas", "fem", "fer", "feu", "fi", "fins", "fora", "gairebé", "ha", "han", "has", "haver", "havia", "he", "hem", "heu", "hi", "ho", "i", "igual", "iguals", "inclòs", "ja", "jo", "l'hi", "la", "les", "li", "li'n", "llarg", "llavors", "m'he", "ma", "mal", "malgrat", "mateix", "mateixa", "mateixes", "mateixos", "me", "mentre", "meu", "meus", "meva", "meves", "mode", "molt", "molta", "moltes", "molts", "mon", "mons", "més", "n'he", "n'hi", "ne", "ni", "no", "nogensmenys", "només", "nosaltres", "nostra", "nostre", "nostres", "o", "oh", "oi", "on", "pas", "pel", "pels", "per", "per que", "perquè", "però", "poc", "poca", "pocs", "podem", "poden", "poder", "podeu", "poques", "potser", "primer", "propi", "puc", "qual", "quals", "quan", "quant", "que", "quelcom", "qui", "quin", "quina", "quines", "quins", "què", "s'ha", "s'han", "sa", "sabem", "saben", "saber", "sabeu", "sap", "saps", "semblant", "semblants", "sense", "ser", "ses", "seu", "seus", "seva", "seves", "si", "sobre", "sobretot", "soc", "solament", "sols", "som", "son", "sons", "sota", "sou", "sóc", "són", "t'ha", "t'han", "t'he", "ta", "tal", "també", "tampoc", "tan", "tant", "tanta", "tantes", "te", "tene", "tenim", "tenir", "teniu", "teu", "teus", "teva", "teves", "tinc", "ton", "tons", "tot", "tota", "totes", "tots", "un", "una", "unes", "uns", "us", "va", "vaig", "vam", "van", "vas", "veu", "vosaltres", "vostra", "vostre", "vostres", "érem", "éreu", "és", "éssent", "últim", "ús"}
STOPWORDS_CA = {"a", "al", "el", "la", "els", "les", "de", "un", "una", "algun", "alguna" } #ha reduit la llista nomes a questes pareules

In [9]:
# Definir función de pre-procesado #tokanitza+filtre elimina stop words
def preprocess(sentence: str) -> List[str]:
    preprocessed = simple_preprocess(sentence)
    preprocessed = [token for token in preprocessed if token not in STOPWORDS_CA]
    return preprocessed

### Entrenament

In [10]:
# Definir constantes de entrenamiento
batch_size: int = 64
num_epochs: int = 64

In [11]:
# Obtener x_train e y_train
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    """
    Otiene las matrices X_1 (N x d) , X_2 (N x d), e Y (n) a partir de listas de parejas de vectores de oraciones - Listas de (d, d, 1)
    :param pair_list:
    :return:
    """
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.array(_x_1), np.array(_x_2)), np.array(_y, dtype=np.float32, )

In [12]:
# Definir el Modelo
def build_and_compile_model(hidden_size: int = 128, embedding_size: int = 300, learning_rate: float = 0.001) -> tf.keras.Model: #ntra vector oracio 1 i 2 + misa embeding
    # Capa de entrada para los pares de vectores
    input_1 = tf.keras.Input(shape=(embedding_size,))
    input_2 = tf.keras.Input(shape=(embedding_size,))

    # Concatenar los pares de vectores
    concatenated = tf.keras.layers.Concatenate()([input_1, input_2])

    concatenated = tf.keras.layers.Dropout(0.1)(concatenated)

    # Capa oculta
    hidden = tf.keras.layers.Dense(hidden_size, activation='tanh')(concatenated)

    # Capa de salida 
    output = tf.keras.layers.Dense(1)(hidden)

    # Definir el modelo con las capas de entrada y salida
    model = tf.keras.Model(inputs=[input_1, input_2], outputs=output)

    # Compilar el modelo
    model.compile(loss='mean_absolute_error',
                  optimizer=tf.keras.optimizers.Adam(learning_rate))

    return model

In [13]:
#Per bag of words i tfidf
def per_tfidf(input_pairs):
    # Preprocesamiento de las oraciones y creación del diccionario  #definir vocabulari
    sentences_1_preproc = [preprocess(sentence_1) for sentence_1, _, _ in input_pairs] 
    sentences_2_preproc = [preprocess(sentence_2) for _, sentence_2, _ in input_pairs]
    sentence_pairs = list(zip(sentences_1_preproc, sentences_2_preproc))
    # Versión aplanada para poder entrenar el modelo
    sentences_pairs_flattened = sentences_1_preproc + sentences_2_preproc #concatena
    diccionario = Dictionary(sentences_pairs_flattened) #constructor (construim un diccionari)
    #cada paraula té associada un índex
    corpus = [diccionario.doc2bow(sent) for sent in sentences_pairs_flattened]
    modelo_tfidf = TfidfModel(corpus) #associar un index de paraula a un pes
    return diccionario, sentences_pairs_flattened,corpus,modelo_tfidf

diccionario, sentences_pairs_flattened,corpus,modelo_tfidf = per_tfidf(dades)

In [206]:
#convertir cada una de les entraddes,frase frase puntaucio a vector vector punt(funció map_pairs) [les frases en forma de vector]
#retorna vector i pes

def map_tf_idf(sentence_preproc: List[str], dictionary: Dictionary, tf_idf_model: TfidfModel) -> Tuple[List[np.ndarray], List[float]]:
    bow = dictionary.doc2bow(sentence_preproc) 
    tf_idf = tf_idf_model[bow] #entrenem el tfidf amb el bag of words obtingut
    vectors, weights = [], []
    for word_index, weight in tf_idf: 
        word = dictionary.get(word_index) #això dona la paraula original
        if word in wv_model:
            vectors.append(wv_model[word]) #vector
            weights.append(weight) #pes
    return vectors, weights

# Función para mapear una frase a su representación one-hot
def map_one_hot(sentence_preproc: List[str], encoder: OneHotEncoder) -> np.ndarray:
    sentence_encoded = encoder.transform(np.array(sentence_preproc).reshape(-1, 1))
    return sentence_encoded

def map_pairs(
        sentence_pairs: List[Tuple[str, str, float]],
        dictionary: Dictionary = None,
        one_hot_model: OneHotEncoder=None,
        tf_idf_model: TfidfModel = None,
        nlp = None,
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    """
    Mapea los tripletes de oraciones a listas de (x, y), (pares de vectores, score)
    :param sentence_pairs:
    :param dictionary:
    :param tf_idf_model:
    :return:
    """
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        sentence_1_preproc = preprocess(sentence_1)  #preproces
        sentence_2_preproc = preprocess(sentence_2)
        # Si usamos TF-IDF
        if tf_idf_model is not None: #es fa servir tf_idf ja que no totes les paraules tenen la mateixa rellevancia en els documents (utilitza els dos tipus de frequencia**)
            # Cálculo del promedio ponderado por TF-IDF de los word embeddings
            vectors1, weights1 = map_tf_idf(sentence_1_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, ) # asociem pes a cada una de les paraules (retorna vectors i pesos)
            vectors2, weights2 = map_tf_idf(sentence_2_preproc, dictionary=dictionary, tf_idf_model=tf_idf_model, )
            if vectors1 == []:
                vector1 = np.zeros((100, ))
            else:
                vector1 = np.average(vectors1, weights=weights1, axis=0, ) #mitjana ponderada
            if vectors2 == []:
                vector2 = np.zeros((100, ))
            else:
                vector2 = np.average(vectors2, weights=weights2, axis=0, )

        else: #opcio 2 -> no fem servir tf_idf
            if one_hot_model is not None:
                # Ajustamos el encoder con todas las palabras en el diccionario
                one_hot_model.fit(np.array(list(dictionary.values())).reshape(-1, 1))
                # Aplicamos OneHotEncoding a las frases preprocesadas
                vectors1 = [map_one_hot(sentence, one_hot_model) for sentence in sentence_1_preproc]
                vectors2 = [map_one_hot(sentence, one_hot_model) for sentence in sentence_2_preproc]
                vector1 = np.sum(vectors1, axis=0) 
                vector2 = np.sum(vectors2, axis=0)
            elif nlp is not None:
                if 'transformer' in nlp.pipe_names:
                    sent1 = nlp(sentence_1)
                    sent2 = nlp(sentence_2)
                    #amb CLS
                    vector1 = sent1._.trf_data.tensors[-1]
                    vector2 = sent2._.trf_data.tensors[-1]
                    #amb mitjana
                    '''matr1 = sent1._.trf_data.tensors[0]
                    matr2 = sent2._.trf_data.tensors[0]
                    vector1 = np.mean(matr1, axis=0)
                    vector2 = np.mean(matr2, axis=0)'''

                else:
                    sent1 = nlp(sentence_1)
                    sent2 = nlp(sentence_2)
                    vectors1 = [sent1[i].vector for i in range(len(sent1)) if str(sent1[i]) in nlp.vocab]
                    vectors2 = [sent2[i].vector for i in range(len(sent2)) if str(sent2[i]) in nlp.vocab]
                    vector1 = np.mean(vectors1, axis=0)
                    vector2 = np.mean(vectors2, axis=0)
            else:
                # Cálculo del promedio de los word embeddings
                vectors1 = [wv_model[word] for word in sentence_1_preproc if word in wv_model]  #obtenim vectors de les dues oracions
                vectors2 = [wv_model[word] for word in sentence_2_preproc if word in wv_model]
                vector1 = np.mean(vectors1, axis=0) #fem mitjana
                vector2 = np.mean(vectors2, axis=0)
                if vectors1 == []:
                    vector1 = np.zeros((100, ))
                if vectors2 == []:
                    vector2 = np.zeros((100, ))
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores
# **
## frequencia directa: numero d'aparicions en el document 
## frequencia inversa: numero de documents en que apareix /numero d'aparicions en el document
## tf_idf fa la multiplicació de tf * idf 

## One Hot

In [21]:
#Convertir a One-Hot
mapped_onehot = map_pairs(input_pairs, tf_idf_model=None, dictionary=diccionario, one_hot_model = OneHotEncoder(sparse=False)) #llsiat de tuples de vector vector, y similitud
mapped_onehot_val = map_pairs(val, tf_idf_model=None, dictionary=diccionario, one_hot_model = OneHotEncoder(sparse=False)) #llsiat de tuples de vector vector, y similitud
mapped_onehot_test = map_pairs(ts,tf_idf_model=None, dictionary=diccionario, one_hot_model = OneHotEncoder(sparse=False)) #llsiat de tuples de vector vector, y similitud

C:\Users\llumf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\llumf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
C:\Users\llumf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\sklearn\preprocessing\_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed

In [29]:
with open('maps_onehot.txt',"w") as f:
    print('TRAIN',file=f)
    print(mapped_onehot,file=f)
    print('VALIDATION',file=f)
    print(mapped_onehot_val,file=f)
    print('TEST',file=f)
    print(mapped_onehot_test,file=f)

In [22]:
x_train_onehot, y_train_onehot = pair_list_to_x_y(mapped_onehot[0:])
x_val_onehot, y_val_onehot = pair_list_to_x_y(mapped_onehot_val[0:])
x_test_onehot, y_test_onehot = pair_list_to_x_y(mapped_onehot_test[0:])
x_test_onehot = (tf.squeeze(x_test_onehot[0], axis=1),tf.squeeze(x_test_onehot[1], axis=1))

In [24]:
# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset_one = tf.data.Dataset.from_tensor_slices((x_train_onehot, y_train_onehot))
train_dataset_one = train_dataset_one.shuffle(buffer_size=len(x_train_onehot)).batch(batch_size)
val_dataset_one = tf.data.Dataset.from_tensor_slices((x_val_onehot, y_val_onehot))
val_dataset_one = val_dataset_one.batch(batch_size)
#test_dataset_one = tf.data.Dataset.from_tensor_slices((x_test_onehot, y_test_onehot))
#test_dataset_one = test_dataset_one.batch(batch_size)

In [25]:
train_dataset_one = train_dataset_one.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))
val_dataset_one = val_dataset_one.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))
#test_dataset_one = test_dataset_one.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [26]:
# Construir y compilar el modelo
model = build_and_compile_model(embedding_size=len(diccionario))

# Entrenar el modelo
model.fit(train_dataset_one, epochs=num_epochs, validation_data=val_dataset_one)

Epoch 1/64
33/33 [==============================] - 3s 58ms/step - loss: 1.3238 - val_loss: 0.9089
Epoch 2/64
33/33 [==============================] - 2s 58ms/step - loss: 0.8413 - val_loss: 0.8856
Epoch 3/64
33/33 [==============================] - 2s 63ms/step - loss: 0.6693 - val_loss: 0.8532
Epoch 4/64
33/33 [==============================] - 2s 63ms/step - loss: 0.5657 - val_loss: 0.9927
Epoch 5/64
33/33 [==============================] - 2s 55ms/step - loss: 0.5724 - val_loss: 0.9451
Epoch 6/64
33/33 [==============================] - 2s 58ms/step - loss: 0.5810 - val_loss: 0.8362
Epoch 7/64
33/33 [==============================] - 2s 57ms/step - loss: 0.5115 - val_loss: 0.8696
Epoch 8/64
33/33 [==============================] - 2s 59ms/step - loss: 0.5305 - val_loss: 0.8211
Epoch 9/64
33/33 [==============================] - 2s 56ms/step - loss: 0.4696 - val_loss: 0.9027
Epoch 10/64
33/33 [==============================] - 2s 59ms/step - loss: 0.4374 - val_loss: 0.9303
Epoch 11/

In [30]:
model.save('mlp_onehot.bin')

INFO:tensorflow:Assets written to: mlp_onehot.bin\assets


INFO:tensorflow:Assets written to: mlp_onehot.bin\assets


In [31]:
from scipy.stats import pearsonr
# Obtener las predicciones del modelo para los datos de prueba. En este ejemplo vamos a utilizar el corpus de training.
y_pred = model.predict(x_test_onehot)
print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_test_onehot.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")


16/16 [==============================] - 0s 6ms/step
[[2.8985887 ]
 [3.1486604 ]
 [1.6135013 ]
 [2.3479908 ]
 [2.3020751 ]
 [2.95383   ]
 [1.8273206 ]
 [2.6824567 ]
 [2.5674171 ]
 [2.5127347 ]
 [2.1076992 ]
 [2.146153  ]
 [2.530677  ]
 [2.1120076 ]
 [3.0523064 ]
 [2.4571414 ]
 [2.756591  ]
 [2.4207056 ]
 [1.7207773 ]
 [2.6669133 ]
 [1.7423093 ]
 [1.6263037 ]
 [1.8231924 ]
 [1.9954944 ]
 [1.7806822 ]
 [1.9790319 ]
 [1.366075  ]
 [2.1870227 ]
 [2.2005222 ]
 [2.7183478 ]
 [1.7238982 ]
 [2.3284452 ]
 [2.2930439 ]
 [2.519018  ]
 [3.1453032 ]
 [1.6507316 ]
 [1.7522564 ]
 [2.422029  ]
 [1.8198595 ]
 [3.3635283 ]
 [2.4597185 ]
 [1.881509  ]
 [2.6133652 ]
 [2.3381736 ]
 [2.0444922 ]
 [3.4486687 ]
 [2.6873996 ]
 [1.3773041 ]
 [1.5094054 ]
 [2.221162  ]
 [2.2095456 ]
 [2.1732333 ]
 [2.5654547 ]
 [0.9261941 ]
 [3.6719825 ]
 [1.7840106 ]
 [2.5781229 ]
 [1.9575707 ]
 [1.6963739 ]
 [2.8109043 ]
 [3.2292912 ]
 [3.2718205 ]
 [2.9408493 ]
 [2.0823271 ]
 [2.8426147 ]
 [2.3144884 ]
 [1.4390489 ]
 [2.27144

In [40]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
from scipy.stats import pearsonr
from scipy import spatial

y_pred_baseline = []

vs1, vs2 = x_test_onehot
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_test_onehot.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.6151985672634542


In [ ]:
modeloss.append("One Hot")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

## Word2vec

In [203]:
from scipy.stats import pearsonr
from scipy import spatial

### 100 MB amb Mean

In [ ]:
#Convertir a vector amb model que tenguem guardat a la variable wv_model 
# Imprimir los pares de vectores y la puntuación de similitud asociada
wv_model = model100.wv
mapped = map_pairs(input_pairs, tf_idf_model=None, dictionary=diccionario, ) 
mapped_val = map_pairs(val, tf_idf_model=None, dictionary=diccionario, )
mapped_ts = map_pairs(ts, tf_idf_model=None, dictionary=diccionario, ) 

In [183]:

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped[0:])
x_val, y_val = pair_list_to_x_y(mapped_val[0:])
x_ts, y_ts = pair_list_to_x_y(mapped_ts[0:])

In [184]:

# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [192]:
# Construir y compilar el modelo
model = build_and_compile_model(embedding_size=100)

# Entrenar el modelo
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

model.save('mlp_wv500.bin')

y_pred = model.predict(x_ts)
print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

Epoch 1/64
33/33 [==============================] - 1s 7ms/step - loss: 1.1450 - val_loss: 0.7751
Epoch 2/64
33/33 [==============================] - 0s 4ms/step - loss: 0.7391 - val_loss: 0.7309
Epoch 3/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6819 - val_loss: 0.7039
Epoch 4/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6575 - val_loss: 0.6908
Epoch 5/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6291 - val_loss: 0.6826
Epoch 6/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6155 - val_loss: 0.6719
Epoch 7/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6079 - val_loss: 0.6617
Epoch 8/64
33/33 [==============================] - 0s 5ms/step - loss: 0.5880 - val_loss: 0.6633
Epoch 9/64
33/33 [==============================] - 0s 4ms/step - loss: 0.5878 - val_loss: 0.6672
Epoch 10/64
33/33 [==============================] - 0s 4ms/step - loss: 0.5594 - val_loss: 0.6578
Epoch 11/64
33/33 [

INFO:tensorflow:Assets written to: mlp_wv500.bin\assets


INFO:tensorflow:Assets written to: mlp_wv500.bin\assets


16/16 [==============================] - 0s 1ms/step
[[2.2739508 ]
 [2.3673146 ]
 [1.4359981 ]
 [1.5908663 ]
 [1.9854666 ]
 [1.9271775 ]
 [2.2739897 ]
 [2.905832  ]
 [1.955568  ]
 [1.5969182 ]
 [2.8184316 ]
 [2.2015784 ]
 [2.2731307 ]
 [2.785246  ]
 [2.4094617 ]
 [2.0931253 ]
 [2.2646925 ]
 [1.6433184 ]
 [2.0652657 ]
 [2.9574664 ]
 [2.4115283 ]
 [2.300415  ]
 [2.3906355 ]
 [2.7070782 ]
 [1.0761046 ]
 [3.1077254 ]
 [2.0539827 ]
 [2.7764661 ]
 [2.6485987 ]
 [1.786126  ]
 [1.7504004 ]
 [2.2407243 ]
 [2.092856  ]
 [1.8178716 ]
 [2.6198995 ]
 [2.4063401 ]
 [1.376235  ]
 [1.9961289 ]
 [2.1967885 ]
 [1.6850477 ]
 [2.767068  ]
 [1.5871361 ]
 [1.6890086 ]
 [3.1906238 ]
 [1.9044989 ]
 [2.5019748 ]
 [1.8137782 ]
 [1.3609943 ]
 [1.8554864 ]
 [1.9581547 ]
 [1.6085895 ]
 [1.5015128 ]
 [2.329975  ]
 [2.0344903 ]
 [3.0213144 ]
 [2.0424654 ]
 [1.4146448 ]
 [2.5826986 ]
 [0.9928692 ]
 [1.2607607 ]
 [2.1909587 ]
 [2.1239712 ]
 [2.940655  ]
 [1.8680505 ]
 [1.8059672 ]
 [1.668193  ]
 [2.9719138 ]
 [2.54035

In [194]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.3579000048133906


In [ ]:
modeloss.append("Word2vec 100MB Mean")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

### 500 MB amb Mean

In [173]:
#Convertir a vector amb model que tenguem guardat a la variable wv_model 
# Imprimir los pares de vectores y la puntuación de similitud asociada
wv_model = model500.wv
mapped500 = map_pairs(input_pairs, tf_idf_model=None, dictionary=diccionario, ) 
mapped_val500 = map_pairs(val, tf_idf_model=None, dictionary=diccionario, )
mapped_ts500 = map_pairs(ts, tf_idf_model=None, dictionary=diccionario, ) 

C:\Users\llumf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\fromnumeric.py:3432: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\llumf\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\numpy\core\_methods.py:190: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [174]:

# Obtener las listas de train y test
x_train500, y_train500 = pair_list_to_x_y(mapped500[0:])
x_val500, y_val500 = pair_list_to_x_y(mapped_val500[0:])
x_ts500, y_ts500 = pair_list_to_x_y(mapped_ts500[0:])

In [177]:

# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset500 = tf.data.Dataset.from_tensor_slices((x_train500, y_train500))
train_dataset500 = train_dataset500.shuffle(buffer_size=len(x_train500)).batch(batch_size)
val_dataset500 = tf.data.Dataset.from_tensor_slices((x_val500, y_val500))
val_dataset500 = val_dataset500.batch(batch_size)

In [179]:
from scipy.stats import pearsonr
# Construir y compilar el modelo
model = build_and_compile_model(embedding_size=100)

# Entrenar el modelo
model.fit(train_dataset500, epochs=num_epochs, validation_data=val_dataset500)

model.save('mlp_wv500.bin')

y_pred = model.predict(x_ts500)
print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts500.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

Epoch 1/64
33/33 [==============================] - 1s 8ms/step - loss: 1.0723 - val_loss: 0.8144
Epoch 2/64
33/33 [==============================] - 0s 4ms/step - loss: 0.7416 - val_loss: 0.7171
Epoch 3/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6864 - val_loss: 0.6854
Epoch 4/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6518 - val_loss: 0.6895
Epoch 5/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6261 - val_loss: 0.6708
Epoch 6/64
33/33 [==============================] - 0s 5ms/step - loss: 0.6082 - val_loss: 0.6768
Epoch 7/64
33/33 [==============================] - 0s 5ms/step - loss: 0.5929 - val_loss: 0.6746
Epoch 8/64
33/33 [==============================] - 0s 7ms/step - loss: 0.5788 - val_loss: 0.6603
Epoch 9/64
33/33 [==============================] - 0s 6ms/step - loss: 0.5660 - val_loss: 0.6740
Epoch 10/64
33/33 [==============================] - 0s 5ms/step - loss: 0.5528 - val_loss: 0.6465
Epoch 11/64
33/33 [

INFO:tensorflow:Assets written to: mlp_wv500.bin\assets


INFO:tensorflow:Assets written to: mlp_wv500.bin\assets


16/16 [==============================] - 0s 2ms/step
[[2.4649055]
 [2.500649 ]
 [2.195281 ]
 [3.3002148]
 [3.0526242]
 [2.7002678]
 [2.0663238]
 [2.6261742]
 [2.0373437]
 [2.2899776]
 [2.9154806]
 [2.591349 ]
 [2.1810584]
 [2.8053985]
 [2.3474884]
 [3.1968136]
 [2.627706 ]
 [2.0658367]
 [2.6566758]
 [2.65123  ]
 [3.0495758]
 [1.4987245]
 [1.9105222]
 [2.4654422]
 [2.085025 ]
 [2.9430134]
 [2.4869454]
 [3.4784834]
 [2.603849 ]
 [3.1957774]
 [2.4116554]
 [2.482859 ]
 [3.2471464]
 [2.568269 ]
 [2.4644113]
 [2.8339262]
 [1.8084263]
 [2.0952246]
 [2.7312932]
 [2.2910647]
 [2.9016066]
 [2.3779032]
 [2.9751558]
 [3.3999968]
 [2.4671874]
 [3.4008565]
 [2.6532822]
 [2.3788948]
 [2.3212824]
 [3.1453357]
 [1.7929403]
 [1.7572   ]
 [1.8190341]
 [2.8694148]
 [2.9987745]
 [3.4790366]
 [2.8082366]
 [3.1029983]
 [2.817199 ]
 [2.6980665]
 [2.2996035]
 [2.3636727]
 [2.7097974]
 [2.5255642]
 [2.6905203]
 [2.7489676]
 [2.649079 ]
 [3.2537227]
 [3.2706158]
 [2.9331214]
 [2.8592458]
 [2.3487148]
 [3.9117346

In [188]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []
vs1, vs2 = x_ts500
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts500.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.36706788610668356


In [ ]:
modeloss.append("Word2vec 500MB Mean")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

### 100MB amb TFIDF

In [207]:
#Convertir a vector amb model que tenguem guardat a la variable wv_model 
# Imprimir los pares de vectores y la puntuación de similitud asociada
wv_model = model100.wv
mapped = map_pairs(input_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, ) 
mapped_val = map_pairs(val, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped_ts = map_pairs(ts, tf_idf_model=modelo_tfidf, dictionary=diccionario, ) 

In [208]:

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped[0:])
x_val, y_val = pair_list_to_x_y(mapped_val[0:])
x_ts, y_ts = pair_list_to_x_y(mapped_ts[0:])

In [209]:

# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [210]:
# Construir y compilar el modelo
model = build_and_compile_model(embedding_size=100)

# Entrenar el modelo
model.fit(train_dataset, epochs=num_epochs, validation_data=val_dataset)

model.save('mlp_wv500.bin')

y_pred = model.predict(x_ts)
print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

Epoch 1/64
33/33 [==============================] - 1s 7ms/step - loss: 1.0516 - val_loss: 0.7688
Epoch 2/64
33/33 [==============================] - 0s 5ms/step - loss: 0.7448 - val_loss: 0.7056
Epoch 3/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6884 - val_loss: 0.6913
Epoch 4/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6608 - val_loss: 0.6875
Epoch 5/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6390 - val_loss: 0.6902
Epoch 6/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6286 - val_loss: 0.6731
Epoch 7/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6103 - val_loss: 0.6733
Epoch 8/64
33/33 [==============================] - 0s 4ms/step - loss: 0.5972 - val_loss: 0.6694
Epoch 9/64
33/33 [==============================] - 0s 4ms/step - loss: 0.5774 - val_loss: 0.6634
Epoch 10/64
33/33 [==============================] - 0s 5ms/step - loss: 0.5721 - val_loss: 0.6557
Epoch 11/64
33/33 [

INFO:tensorflow:Assets written to: mlp_wv500.bin\assets


INFO:tensorflow:Assets written to: mlp_wv500.bin\assets


16/16 [==============================] - 0s 3ms/step
[[2.185686 ]
 [2.365426 ]
 [2.2443397]
 [2.7123919]
 [2.8816934]
 [2.4265723]
 [2.7577012]
 [2.6047997]
 [2.4269898]
 [1.9284688]
 [3.1075478]
 [3.1381333]
 [2.3651843]
 [2.6801684]
 [2.98719  ]
 [2.4454148]
 [2.9014966]
 [3.5822694]
 [2.2938037]
 [2.9659185]
 [2.702299 ]
 [1.9663504]
 [2.368576 ]
 [1.8258245]
 [1.6361303]
 [2.366942 ]
 [2.5087082]
 [2.3580396]
 [2.3068483]
 [2.846452 ]
 [2.6018443]
 [2.8989136]
 [2.878146 ]
 [2.08905  ]
 [2.8367827]
 [2.5641503]
 [2.540885 ]
 [2.3480988]
 [2.331429 ]
 [2.3022404]
 [2.8759933]
 [1.9699187]
 [2.6063766]
 [2.606659 ]
 [2.6346426]
 [3.0175085]
 [2.2572455]
 [1.168063 ]
 [1.9680239]
 [3.6533842]
 [2.2211435]
 [2.0859008]
 [2.4107242]
 [1.9788076]
 [3.5959249]
 [2.3657634]
 [2.4003677]
 [2.6868293]
 [1.6635345]
 [2.5856276]
 [1.8474929]
 [2.7142062]
 [3.2993183]
 [2.2232852]
 [2.347542 ]
 [2.1385672]
 [2.5567808]
 [3.0450277]
 [2.1354523]
 [2.3364625]
 [2.3960798]
 [2.1641188]
 [3.6058128

In [211]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.3662921654596845


In [ ]:
modeloss.append("Word2vec 100MB TFIDF")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

### 500 MB amb TFIDF

In [212]:
#Convertir a vector amb model que tenguem guardat a la variable wv_model 
# Imprimir los pares de vectores y la puntuación de similitud asociada
wv_model = model500.wv
mapped500 = map_pairs(input_pairs, tf_idf_model=modelo_tfidf, dictionary=diccionario, ) 
mapped_val500 = map_pairs(val, tf_idf_model=modelo_tfidf, dictionary=diccionario, )
mapped_ts500 = map_pairs(ts, tf_idf_model=modelo_tfidf, dictionary=diccionario, ) 

In [213]:

# Obtener las listas de train y test
x_train500, y_train500 = pair_list_to_x_y(mapped500[0:])
x_val500, y_val500 = pair_list_to_x_y(mapped_val500[0:])
x_ts500, y_ts500 = pair_list_to_x_y(mapped_ts500[0:])

In [214]:

# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset500 = tf.data.Dataset.from_tensor_slices((x_train500, y_train500))
train_dataset500 = train_dataset500.shuffle(buffer_size=len(x_train500)).batch(batch_size)
val_dataset500 = tf.data.Dataset.from_tensor_slices((x_val500, y_val500))
val_dataset500 = val_dataset500.batch(batch_size)

In [215]:
from scipy.stats import pearsonr
# Construir y compilar el modelo
model = build_and_compile_model(embedding_size=100)

# Entrenar el modelo
model.fit(train_dataset500, epochs=num_epochs, validation_data=val_dataset500)

model.save('mlp_wv500.bin')

y_pred = model.predict(x_ts500)
print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts500.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

Epoch 1/64
33/33 [==============================] - 1s 7ms/step - loss: 1.3019 - val_loss: 0.8194
Epoch 2/64
33/33 [==============================] - 0s 5ms/step - loss: 0.7757 - val_loss: 0.7197
Epoch 3/64
33/33 [==============================] - 0s 4ms/step - loss: 0.7064 - val_loss: 0.7076
Epoch 4/64
33/33 [==============================] - 0s 4ms/step - loss: 0.6621 - val_loss: 0.6793
Epoch 5/64
33/33 [==============================] - 0s 8ms/step - loss: 0.6350 - val_loss: 0.6698
Epoch 6/64
33/33 [==============================] - 0s 5ms/step - loss: 0.6174 - val_loss: 0.6660
Epoch 7/64
33/33 [==============================] - 0s 5ms/step - loss: 0.6034 - val_loss: 0.6666
Epoch 8/64
33/33 [==============================] - 0s 5ms/step - loss: 0.5783 - val_loss: 0.6662
Epoch 9/64
33/33 [==============================] - 0s 5ms/step - loss: 0.5643 - val_loss: 0.6669
Epoch 10/64
33/33 [==============================] - 0s 5ms/step - loss: 0.5514 - val_loss: 0.6746
Epoch 11/64
33/33 [

INFO:tensorflow:Assets written to: mlp_wv500.bin\assets


INFO:tensorflow:Assets written to: mlp_wv500.bin\assets


16/16 [==============================] - 0s 2ms/step
[[2.9419417]
 [2.6210876]
 [2.3655107]
 [2.2998486]
 [2.965015 ]
 [3.2385285]
 [2.9847474]
 [3.099049 ]
 [2.8623822]
 [2.8020535]
 [3.3783054]
 [3.080738 ]
 [2.8081076]
 [3.4534883]
 [3.3354206]
 [2.8396785]
 [3.6171417]
 [2.7610464]
 [2.110979 ]
 [3.7025986]
 [3.2693343]
 [2.1890154]
 [2.5548053]
 [2.6900501]
 [1.954354 ]
 [2.549803 ]
 [2.7923563]
 [3.5862827]
 [2.8753333]
 [3.568235 ]
 [2.3792617]
 [2.9960132]
 [2.4484017]
 [2.9330602]
 [3.6868925]
 [2.746467 ]
 [2.145937 ]
 [2.8144093]
 [2.271559 ]
 [2.9137807]
 [2.7308288]
 [2.0753841]
 [3.4311976]
 [2.1963115]
 [3.8251967]
 [4.0534663]
 [1.3535678]
 [2.7695575]
 [2.3000817]
 [1.2819825]
 [1.6903087]
 [2.8906803]
 [3.1450968]
 [2.721918 ]
 [3.0926192]
 [2.88858  ]
 [2.8680592]
 [3.4546404]
 [2.7542093]
 [3.2505486]
 [2.0466063]
 [3.5447843]
 [3.8600187]
 [2.8307796]
 [3.5400283]
 [2.674286 ]
 [2.164058 ]
 [2.8216567]
 [3.1683807]
 [3.0055447]
 [3.2231083]
 [1.9558685]
 [3.6047602

In [217]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []
vs1, vs2 = x_ts500
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts500.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}") 

Correlación de Pearson: 0.3851572899725914


In [ ]:
modeloss.append("Word2vec 500MB TFIDF")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

## spaCy

In [22]:
from scipy.stats import pearsonr

In [31]:
import spacy
nlp = spacy.load("ca_core_news_sm")


In [32]:
mapped_spacy = map_pairs(input_pairs,nlp=nlp)
mapped_spacy_val = map_pairs(val,nlp=nlp)
mapped_spacy_test = map_pairs(ts,nlp=nlp)

In [218]:
# Obtener las listas de train y test
x_train_spacy, y_train_spacy = pair_list_to_x_y(mapped_spacy[0:])
x_val_spacy, y_val_spacy = pair_list_to_x_y(mapped_spacy_val[0:])
x_ts_spacy, y_ts_spacy = pair_list_to_x_y(mapped_spacy_test[0:])

# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset_spacy = tf.data.Dataset.from_tensor_slices((x_train_spacy, y_train_spacy))
train_dataset_spacy = train_dataset_spacy.shuffle(buffer_size=len(x_train_spacy)).batch(batch_size)
val_dataset_spacy = tf.data.Dataset.from_tensor_slices((x_val_spacy, y_val_spacy))
val_dataset_spacy = val_dataset_spacy.batch(batch_size)

In [219]:

# Construir y compilar el modelo
model = build_and_compile_model(embedding_size=96)

# Entrenar el modelo
model.fit(train_dataset_spacy, epochs=num_epochs, validation_data=val_dataset_spacy)

model.save('mlp_spacy.bin')

y_pred = model.predict(x_ts_spacy)
print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts_spacy.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

Epoch 1/64
33/33 [==============================] - 1s 16ms/step - loss: 1.0482 - val_loss: 0.7734
Epoch 2/64
33/33 [==============================] - 0s 6ms/step - loss: 0.7232 - val_loss: 0.7073
Epoch 3/64
33/33 [==============================] - 0s 6ms/step - loss: 0.6821 - val_loss: 0.6931
Epoch 4/64
33/33 [==============================] - 0s 7ms/step - loss: 0.6631 - val_loss: 0.6859
Epoch 5/64
33/33 [==============================] - 0s 6ms/step - loss: 0.6531 - val_loss: 0.6838
Epoch 6/64
33/33 [==============================] - 0s 5ms/step - loss: 0.6431 - val_loss: 0.6837
Epoch 7/64
33/33 [==============================] - 0s 6ms/step - loss: 0.6370 - val_loss: 0.6819
Epoch 8/64
33/33 [==============================] - 0s 7ms/step - loss: 0.6297 - val_loss: 0.6740
Epoch 9/64
33/33 [==============================] - 0s 6ms/step - loss: 0.6179 - val_loss: 0.6704
Epoch 10/64
33/33 [==============================] - 0s 6ms/step - loss: 0.6094 - val_loss: 0.6754
Epoch 11/64
33/33 

INFO:tensorflow:Assets written to: mlp_spacy.bin\assets


INFO:tensorflow:Assets written to: mlp_spacy.bin\assets


16/16 [==============================] - 0s 2ms/step
[[2.605318  ]
 [2.8917828 ]
 [1.7647489 ]
 [2.4151971 ]
 [2.4446979 ]
 [2.6809394 ]
 [2.1797233 ]
 [3.0993328 ]
 [2.2394166 ]
 [1.8022718 ]
 [2.8417783 ]
 [2.2890208 ]
 [2.4153225 ]
 [2.773149  ]
 [2.1507585 ]
 [2.2853942 ]
 [2.7306376 ]
 [2.2136862 ]
 [2.6330185 ]
 [2.3184824 ]
 [1.8707652 ]
 [1.6519599 ]
 [2.5710378 ]
 [2.3305526 ]
 [1.0181341 ]
 [2.300779  ]
 [2.1430721 ]
 [2.8753147 ]
 [3.0484767 ]
 [2.2803583 ]
 [2.75098   ]
 [1.9387451 ]
 [2.0392263 ]
 [2.199273  ]
 [2.6368055 ]
 [3.1218204 ]
 [2.7563457 ]
 [2.811563  ]
 [2.4408975 ]
 [2.233152  ]
 [2.869747  ]
 [2.1177702 ]
 [2.2877986 ]
 [2.6774473 ]
 [2.332977  ]
 [3.4336786 ]
 [2.6058207 ]
 [1.6507437 ]
 [1.9793435 ]
 [2.846611  ]
 [2.4446533 ]
 [3.2940888 ]
 [2.4641962 ]
 [2.9835656 ]
 [2.5427814 ]
 [2.4062634 ]
 [2.6389823 ]
 [2.6150947 ]
 [2.4639192 ]
 [2.4909954 ]
 [2.6825895 ]
 [3.129556  ]
 [2.8638813 ]
 [2.5312386 ]
 [2.2596717 ]
 [2.069059  ]
 [1.6646442 ]
 [2.86535

In [221]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
from scipy.stats import pearsonr
from scipy import spatial

y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts_spacy.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.3662921654596845


In [ ]:
modeloss.append("spaCy")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

## RoBERTa

In [15]:
import spacy
nlp = spacy.load("ca_core_news_trf")

In [28]:
mapped_spacytrf = map_pairs(input_pairs,nlp=nlp)
mapped_spacytrf_val = map_pairs(val,nlp=nlp)
mapped_spacytrf_test = map_pairs(ts,nlp=nlp)

In [224]:
# Obtener las listas de train y test
x_train_spacy, y_train_spacy = pair_list_to_x_y(mapped_spacytrf[0:])
x_val_spacy, y_val_spacy = pair_list_to_x_y(mapped_spacytrf_val[0:])
x_ts_spacy, y_ts_spacy = pair_list_to_x_y(mapped_spacytrf_test[0:])
x_ts_spacy = (tf.squeeze(x_ts_spacy[0], axis=1),tf.squeeze(x_ts_spacy[1], axis=1))

# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset_spacy = tf.data.Dataset.from_tensor_slices((x_train_spacy, y_train_spacy))
train_dataset_spacy = train_dataset_spacy.shuffle(buffer_size=len(x_train_spacy)).batch(batch_size)
val_dataset_spacy = tf.data.Dataset.from_tensor_slices((x_val_spacy, y_val_spacy))
val_dataset_spacy = val_dataset_spacy.batch(batch_size)

train_dataset_spacy = train_dataset_spacy.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))
val_dataset_spacy = val_dataset_spacy.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))
#test_dataset_one = test_dataset_one.map(lambda x, y: ((tf.squeeze(x[0], axis=1), tf.squeeze(x[1], axis=1)), y))


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [225]:

# Construir y compilar el modelo
model = build_and_compile_model(embedding_size=768)

# Entrenar el modelo
model.fit(train_dataset_spacy, epochs=num_epochs, validation_data=val_dataset_spacy)

model.save('mlp_spacytrf.bin')

y_pred = model.predict(x_ts_spacy)
print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts_spacy.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

Epoch 1/64
33/33 [==============================] - 1s 9ms/step - loss: 1.0913 - val_loss: 0.7408
Epoch 2/64
33/33 [==============================] - 0s 7ms/step - loss: 0.7007 - val_loss: 0.7118
Epoch 3/64
33/33 [==============================] - 0s 7ms/step - loss: 0.6689 - val_loss: 0.6746
Epoch 4/64
33/33 [==============================] - 0s 7ms/step - loss: 0.6583 - val_loss: 0.6746
Epoch 5/64
33/33 [==============================] - 0s 7ms/step - loss: 0.6490 - val_loss: 0.6684
Epoch 6/64
33/33 [==============================] - 0s 8ms/step - loss: 0.6631 - val_loss: 0.6662
Epoch 7/64
33/33 [==============================] - 0s 7ms/step - loss: 0.6541 - val_loss: 0.6912
Epoch 8/64
33/33 [==============================] - 0s 7ms/step - loss: 0.6537 - val_loss: 0.6707
Epoch 9/64
33/33 [==============================] - 0s 7ms/step - loss: 0.6420 - val_loss: 0.6541
Epoch 10/64
33/33 [==============================] - 0s 7ms/step - loss: 0.6290 - val_loss: 0.6565
Epoch 11/64
33/33 [

INFO:tensorflow:Assets written to: mlp_spacytrf.bin\assets


INFO:tensorflow:Assets written to: mlp_spacytrf.bin\assets


16/16 [==============================] - 0s 2ms/step
[[2.69529  ]
 [2.5506654]
 [2.11744  ]
 [2.7189069]
 [1.8581902]
 [2.632073 ]
 [2.87057  ]
 [2.7373865]
 [2.2718265]
 [2.4003034]
 [2.535298 ]
 [2.8960884]
 [2.6907573]
 [2.612858 ]
 [2.4195418]
 [2.4544556]
 [2.5884998]
 [2.4280834]
 [2.3461413]
 [2.6328084]
 [2.7264643]
 [2.2664874]
 [1.9033583]
 [1.7881253]
 [2.3935711]
 [2.2634008]
 [1.7027473]
 [2.7918384]
 [2.2954636]
 [2.509907 ]
 [2.4430444]
 [2.1552742]
 [2.852555 ]
 [2.5041926]
 [2.8107657]
 [2.4417171]
 [2.482497 ]
 [2.708228 ]
 [2.0431035]
 [2.054243 ]
 [2.8010564]
 [2.7122266]
 [2.6922703]
 [2.7063217]
 [2.5216668]
 [2.762389 ]
 [2.7370093]
 [2.0094173]
 [1.9582055]
 [2.809252 ]
 [2.4961014]
 [2.6789672]
 [2.5482562]
 [2.931455 ]
 [2.7213295]
 [2.7473867]
 [2.369176 ]
 [2.403814 ]
 [2.5996728]
 [2.609829 ]
 [2.8241239]
 [2.6231596]
 [2.4965618]
 [2.2214088]
 [2.483448 ]
 [2.457045 ]
 [2.0521333]
 [2.7344027]
 [2.564619 ]
 [2.7114856]
 [2.7250998]
 [2.657009 ]
 [2.8794458

In [226]:
y_pred_baseline = []

vs1, vs2 = x_ts_spacy
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlation, _ = pearsonr(np.array(y_pred_baseline), y_ts_spacy.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlation}E") E

Correlación de Pearson: 0.18147380490469453


In [ ]:
modeloss.append("RoBERTa")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

## RoBERTa fine-tuned

In [245]:
test_x = []
test_y = []
for x in ts:
    test_x.append((x[0],x[1]))
    test_y.append(x[2])

In [246]:
from transformers import pipeline, AutoTokenizer
from scipy.special import logit

model = 'projecte-aina/roberta-base-ca-v2-cased-sts'
tokenizer = AutoTokenizer.from_pretrained(model)
pipe = pipeline('text-classification', model=model, tokenizer=tokenizer)

def prepare(sentence_pairs):
    sentence_pairs_prep = []
    for s1, s2 in sentence_pairs:
        sentence_pairs_prep.append(f"{tokenizer.cls_token} {s1}{tokenizer.sep_token}{tokenizer.sep_token} {s2}{tokenizer.sep_token}")
    return sentence_pairs_prep

sentence_pairs = test_x.copy()

predictions = pipe(prepare(sentence_pairs), add_special_tokens=False)
y_pred = []
y_predict = []
# convert back to scores to the original 0 and 5 interval
for prediction in predictions:
    y_predict.append(logit(prediction['score']))
    prediction['score'] = logit(prediction['score'])
print(predictions)

[{'label': 'SIMILARITY', 'score': 2.1410346445880513}, {'label': 'SIMILARITY', 'score': 3.0859965063408987}, {'label': 'SIMILARITY', 'score': 1.6586843002953948}, {'label': 'SIMILARITY', 'score': 3.2563704928342947}, {'label': 'SIMILARITY', 'score': 2.4166337310374097}, {'label': 'SIMILARITY', 'score': 3.4185356584153817}, {'label': 'SIMILARITY', 'score': 2.838414066682235}, {'label': 'SIMILARITY', 'score': 3.782994356337281}, {'label': 'SIMILARITY', 'score': 2.4721758212776597}, {'label': 'SIMILARITY', 'score': 2.158547000649474}, {'label': 'SIMILARITY', 'score': 1.6486324447085778}, {'label': 'SIMILARITY', 'score': 2.378597435771381}, {'label': 'SIMILARITY', 'score': 2.6842834808452993}, {'label': 'SIMILARITY', 'score': 3.0824030060422962}, {'label': 'SIMILARITY', 'score': 1.9718790901504786}, {'label': 'SIMILARITY', 'score': 3.6191161980727538}, {'label': 'SIMILARITY', 'score': 2.4404707091749747}, {'label': 'SIMILARITY', 'score': 1.9142290836862115}, {'label': 'SIMILARITY', 'score'

In [263]:
correlationM, _ = pearsonr(y_predict, test_y)
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), test_y)
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}")

Correlación de Pearson: 0.7819885797624594
Correlación de Pearson: 0.3662921654596845


In [ ]:
modeloss.append("RoBERTa fine-tuned")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

# Part 3: sequence model

In [266]:
# Tipado
from typing import Optional

## Funcions

In [267]:
REMAP_EMBEDDINGS: bool = True
USE_PRETRAINED: bool = True

In [268]:
def map_word_embeddings(
        sentence: str,
        sequence_len: int = 32,
        fixed_dictionary: Optional[Dictionary] = None
) -> np.ndarray:
    """
    Map to word-embedding indices
    :param sentence:
    :param sequence_len:
    :param fixed_dictionary:
    :return:
    """
    sentence_preproc = simple_preprocess(sentence)
    _vectors = np.zeros(sequence_len, dtype=np.int32)
    index = 0
    for word in sentence_preproc:
        if fixed_dictionary is not None:
            if word in fixed_dictionary.token2id:
                # Sumo 1 porque el valor 0 está reservado a padding
                _vectors[index] = fixed_dictionary.token2id[word] + 1
                index += 1
        else:
            if word in wv_model.key_to_index:
                _vectors[index] = wv_model.key_to_index[word] + 1
                index += 1
    return _vectors


def map_pairs(
        sentence_pairs: List[Tuple[str, str, float]],
        sequence_len: int = 32,
        fixed_dictionary: Optional[Dictionary] = None
) -> List[Tuple[Tuple[np.ndarray, np.ndarray], float]]:
    """
    Mapea los tripletes de oraciones a listas de (x, y), (pares de vectores, score)
    :param sentence_pairs:
    :param sequence_len:
    :param fixed_dictionary:
    :return:
    """
    # Mapeo de los pares de oraciones a pares de vectores
    pares_vectores = []
    for i, (sentence_1, sentence_2, similitud) in enumerate(sentence_pairs):
        vector1 = map_word_embeddings(sentence_1, sequence_len, fixed_dictionary)
        vector2 = map_word_embeddings(sentence_2, sequence_len, fixed_dictionary)
        # Añadir a la lista
        pares_vectores.append(((vector1, vector2), similitud))
    return pares_vectores

In [277]:
# Definir el Modelo
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
def build_and_compile_model(
        input_length: int = 32,
        hidden_size: int = 64,
        dictionary_size: int = 1000,
        embedding_size: int = 16,
        pretrained_weights: Optional[np.ndarray] = None,
        learning_rate: float = 0.001,
        trainable: bool = False,
) -> tf.keras.Model:
    """
    Este es un modelo muy básico. Hace lo mismo que el modelo single_vector. La puntuación es mejor por no eliminar stopwords.
    :param input_length:
    :param hidden_size:
    :param dictionary_size:
    :param embedding_size:
    :param pretrained_weights:
    :param learning_rate:
    :param trainable:
    :return:
    """
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32, ), tf.keras.Input((input_length, ), dtype=tf.int32, )
    # Define Layers
    if pretrained_weights is None:
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True, )
    else:
        dictionary_size = pretrained_weights.shape[0]
        embedding_size = pretrained_weights.shape[1]
        initializer = tf.keras.initializers.Constant(pretrained_weights)
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True,
            embeddings_initializer=initializer, trainable=trainable, )
    pooling = tf.keras.layers.GlobalAveragePooling1D()
    # Pass through the layers
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    _embedded_1, _embedded_2 = embedding(input_1, ), embedding(input_2, )
    _pooled_1, _pooled_2 = pooling(_embedded_1, mask=_input_mask_1), pooling(_embedded_2, mask=_input_mask_2)

    # Compute the cosine distance
    projected_1 = tf.linalg.l2_normalize(_pooled_1, axis=1, )
    projected_2 = tf.linalg.l2_normalize(_pooled_2, axis=1, )
    output = 2.5 * (1.0 + tf.reduce_sum(projected_1 * projected_2, axis=1, ))

    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2, ), outputs=output,)
    model.compile(loss='mean_squared_error',
                optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    return model

In [299]:
# Definir el Modelo
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
def build_and_compile_model_adamax(
        input_length: int = 32,
        hidden_size: int = 64,
        dictionary_size: int = 1000,
        embedding_size: int = 16,
        pretrained_weights: Optional[np.ndarray] = None,
        learning_rate: float = 0.001,
        trainable: bool = False,
) -> tf.keras.Model:
    """
    Este es un modelo muy básico. Hace lo mismo que el modelo single_vector. La puntuación es mejor por no eliminar stopwords.
    :param input_length:
    :param hidden_size:
    :param dictionary_size:
    :param embedding_size:
    :param pretrained_weights:
    :param learning_rate:
    :param trainable:
    :return:
    """
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32, ), tf.keras.Input((input_length, ), dtype=tf.int32, )
    # Define Layers
    if pretrained_weights is None:
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True, )
    else:
        dictionary_size = pretrained_weights.shape[0]
        embedding_size = pretrained_weights.shape[1]
        initializer = tf.keras.initializers.Constant(pretrained_weights)
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True,
            embeddings_initializer=initializer, trainable=trainable, )
    pooling = tf.keras.layers.GlobalAveragePooling1D()
    # Pass through the layers
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    _embedded_1, _embedded_2 = embedding(input_1, ), embedding(input_2, )
    _pooled_1, _pooled_2 = pooling(_embedded_1, mask=_input_mask_1), pooling(_embedded_2, mask=_input_mask_2)

    # Compute the cosine distance
    projected_1 = tf.linalg.l2_normalize(_pooled_1, axis=1, )
    projected_2 = tf.linalg.l2_normalize(_pooled_2, axis=1, )
    output = 2.5 * (1.0 + tf.reduce_sum(projected_1 * projected_2, axis=1, ))

    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2, ), outputs=output,)
    model.compile(loss='mean_squared_error',
                optimizer=tf.keras.optimizers.Adamax(learning_rate=learning_rate))
    return model

In [303]:
# Definir el Modelo
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')
def build_and_compile_model_reg(
        input_length: int = 32,
        hidden_size: int = 64,
        dictionary_size: int = 1000,
        embedding_size: int = 16,
        pretrained_weights: Optional[np.ndarray] = None,
        learning_rate: float = 0.001,
        trainable: bool = False,
        embeddings_regularizer = None
) -> tf.keras.Model:
    """
    Este es un modelo muy básico. Hace lo mismo que el modelo single_vector. La puntuación es mejor por no eliminar stopwords.
    :param input_length:
    :param hidden_size:
    :param dictionary_size:
    :param embedding_size:
    :param pretrained_weights:
    :param learning_rate:
    :param trainable:
    :return:
    """
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32, ), tf.keras.Input((input_length, ), dtype=tf.int32, )
    # Define Layers
    if pretrained_weights is None:
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True, )
    else:
        dictionary_size = pretrained_weights.shape[0]
        embedding_size = pretrained_weights.shape[1]
        initializer = tf.keras.initializers.Constant(pretrained_weights)
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True,
            embeddings_initializer=initializer, trainable=trainable, embeddings_regularizer=embeddings_regularizer, )
    pooling = tf.keras.layers.GlobalAveragePooling1D()
    # Pass through the layers
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    _embedded_1, _embedded_2 = embedding(input_1, ), embedding(input_2, )
    _pooled_1, _pooled_2 = pooling(_embedded_1, mask=_input_mask_1), pooling(_embedded_2, mask=_input_mask_2)

    # Compute the cosine distance
    projected_1 = tf.linalg.l2_normalize(_pooled_1, axis=1, )
    projected_2 = tf.linalg.l2_normalize(_pooled_2, axis=1, )
    output = 2.5 * (1.0 + tf.reduce_sum(projected_1 * projected_2, axis=1, ))

    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2, ), outputs=output,)
    model.compile(loss='mean_squared_error',
                optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    return model

In [270]:
# Definir el Modelo
import tensorflow as tf
def build_and_compile_model_2(
        input_length: int = 32,
        dictionary_size: int = 1000,
        embedding_size: int = 16,
        pretrained_weights: Optional[np.ndarray] = None,
        learning_rate: float = 0.001,
        trainable: bool = False,
) -> tf.keras.Model:
    """
    Este es un modelo algo más avanzado. Calcula internamente una media ponderada de los word embeddings. Calcula también la proyección.
    :param input_length:
    :param hidden_size:
    :param dictionary_size:
    :param embedding_size:
    :param pretrained_weights:
    :param learning_rate:
    :param trainable:
    :return:
    """
    input_1, input_2 = tf.keras.Input((input_length, ), dtype=tf.int32, ), tf.keras.Input((input_length, ), dtype=tf.int32, )
    # Define Layers
    if pretrained_weights is None:
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True, )
    else:
        dictionary_size = pretrained_weights.shape[0]
        embedding_size = pretrained_weights.shape[1]
        initializer = tf.keras.initializers.Constant(pretrained_weights)
        embedding = tf.keras.layers.Embedding(
            dictionary_size, embedding_size, input_length=input_length, mask_zero=True,
            embeddings_initializer=initializer, trainable=trainable, )
    # Pass through the layers
    _input_mask_1, _input_mask_2 = tf.not_equal(input_1, 0), tf.not_equal(input_2, 0)
    _embedded_1, _embedded_2 = embedding(input_1, ), embedding(input_2, )

    # Compute custom weights
    weights_computation = tf.keras.layers.Dense(1, name="weight_computation")
    dropout = tf.keras.layers.Dropout(0.2, name="dropout_in")
    _weights_1 = weights_computation(dropout(_embedded_1))
    weights_1 = tf.squeeze(_weights_1, axis=[-1])
    _weights_2 = weights_computation(dropout(_embedded_2))
    weights_2 = tf.squeeze(_weights_2, axis=[-1])
    # Define softmax
    softmax = tf.keras.layers.Softmax(name="weighted_sum_softmax")
    scores_1 = softmax(weights_1, mask=_input_mask_1)
    _pooled_1 = tf.math.reduce_sum(_embedded_1 * tf.expand_dims(scores_1, axis=-1), axis=1)
    scores_2 = softmax(weights_2, mask=_input_mask_2)
    _pooled_2 = tf.math.reduce_sum(_embedded_2 * tf.expand_dims(scores_2, axis=-1) , axis=1)
    # Compute the distance
    dense_output = tf.keras.layers.Dense(1)
    dropout_out = tf.keras.layers.Dropout(0.2, name="dropout_out")
    projected_1 = tf.linalg.l2_normalize(_pooled_1, axis=1, )
    projected_2 = tf.linalg.l2_normalize(_pooled_2, axis=1, )
    output = dense_output(dropout_out(projected_1 * projected_2), )

    # Define the model
    model = tf.keras.Model(inputs=(input_1, input_2, ), outputs=output,)
    model.compile(loss='mean_absolute_error',
                optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate))
    return model

In [271]:
def pair_list_to_x_y(pair_list: List[Tuple[Tuple[np.ndarray, np.ndarray], int]]) -> Tuple[Tuple[np.ndarray, np.ndarray], np.ndarray]:
    """
    Otiene las matrices X_1 (N x d) , X_2 (N x d), e Y (n) a partir de listas de parejas de vectores de oraciones - Listas de (d, d, 1)
    :param pair_list:
    :return:
    """
    _x, _y = zip(*pair_list)
    _x_1, _x_2 = zip(*_x)
    return (np.row_stack(_x_1), np.row_stack(_x_2)), np.array(_y)

## 100 MB amb Mean

In [279]:
#Convertir a vector amb model que tenguem guardat a la variable wv_model 
# Imprimir los pares de vectores y la puntuación de similitud asociada
wv_model = model100.wv
mapped = map_pairs(input_pairs, fixed_dictionary=diccionario, ) 
mapped_val = map_pairs(val, fixed_dictionary=diccionario, )
mapped_ts = map_pairs(ts, fixed_dictionary=diccionario, ) 

In [275]:

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped[0:])
x_val, y_val = pair_list_to_x_y(mapped_val[0:])
x_ts, y_ts = pair_list_to_x_y(mapped_ts[0:])

In [276]:

# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [280]:
pretrained_weights: Optional[np.ndarray] = None
if USE_PRETRAINED:
    if REMAP_EMBEDDINGS:
        pretrained_weights = np.zeros(
            (len(diccionario.token2id) + 1, wv_model.vector_size),  dtype=np.float32)
        for token, _id in diccionario.token2id.items():
            if token in wv_model:
                pretrained_weights[_id + 1] = wv_model[token]
            else:
                # In W2V, OOV will not have a representation. We will use 0.
                pass
    else:
        # Not recommended (this will consume A LOT of RAM)
        pretrained_weights = np.zeros((wv_model.vectors.shape[0] + 1, wv_model.vector_size,),  dtype=np.float32)
        pretrained_weights[1:, :] = wv_model.vectors

### Model 1

In [285]:
# Construir y compilar el modelo
#Hem augmentat el learning rate per a disminuir més ràpidament la loss i poder reduir el nombre d'epochs. Així faria overfitting
model = build_and_compile_model(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=32, validation_data=val_dataset)

Epoch 1/32
33/33 [==============================] - 2s 19ms/step - loss: 4.3234 - val_loss: 4.1764
Epoch 2/32
33/33 [==============================] - 0s 15ms/step - loss: 3.8426 - val_loss: 4.0016
Epoch 3/32
33/33 [==============================] - 1s 15ms/step - loss: 3.3471 - val_loss: 3.8308
Epoch 4/32
33/33 [==============================] - 1s 16ms/step - loss: 2.8443 - val_loss: 3.6738
Epoch 5/32
33/33 [==============================] - 1s 16ms/step - loss: 2.3673 - val_loss: 3.5362
Epoch 6/32
33/33 [==============================] - 1s 16ms/step - loss: 1.9390 - val_loss: 3.4183
Epoch 7/32
33/33 [==============================] - 1s 17ms/step - loss: 1.5673 - val_loss: 3.3177
Epoch 8/32
33/33 [==============================] - 1s 20ms/step - loss: 1.2514 - val_loss: 3.2330
Epoch 9/32
33/33 [==============================] - 1s 16ms/step - loss: 0.9923 - val_loss: 3.1626
Epoch 10/32
33/33 [==============================] - 1s 16ms/step - loss: 0.7851 - val_loss: 3.1053
Epoch 11/

In [286]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.4422017787406168


In [287]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


### Model 2

In [318]:
#Provem el segon model
model = build_and_compile_model_2(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=64, validation_data=val_dataset)

Epoch 1/64
33/33 [==============================] - 3s 32ms/step - loss: 2.2909 - val_loss: 2.0126
Epoch 2/64
33/33 [==============================] - 1s 22ms/step - loss: 1.7397 - val_loss: 1.4488
Epoch 3/64
33/33 [==============================] - 1s 22ms/step - loss: 1.1643 - val_loss: 0.9326
Epoch 4/64
33/33 [==============================] - 1s 19ms/step - loss: 0.7795 - val_loss: 0.6988
Epoch 5/64
33/33 [==============================] - 1s 18ms/step - loss: 0.6393 - val_loss: 0.6422
Epoch 6/64
33/33 [==============================] - 1s 18ms/step - loss: 0.5649 - val_loss: 0.6322
Epoch 7/64
33/33 [==============================] - 1s 22ms/step - loss: 0.5050 - val_loss: 0.6161
Epoch 8/64
33/33 [==============================] - 1s 21ms/step - loss: 0.4649 - val_loss: 0.6253
Epoch 9/64
33/33 [==============================] - 1s 24ms/step - loss: 0.4218 - val_loss: 0.6252
Epoch 10/64
33/33 [==============================] - 1s 26ms/step - loss: 0.3820 - val_loss: 0.6220
Epoch 11/

In [317]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.24598748743437876


In [298]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


In [ ]:
modeloss.append("Word2vec 100MB Mean")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

### Model adamax
Provem amb un optimitzador diferent per veure si obtenim millors resultats

In [300]:
#Provem el segon model
model = build_and_compile_model_adamax(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=64, validation_data=val_dataset)

Epoch 1/64
33/33 [==============================] - 2s 18ms/step - loss: 4.3603 - val_loss: 4.2699
Epoch 2/64
33/33 [==============================] - 0s 13ms/step - loss: 4.2171 - val_loss: 4.2123
Epoch 3/64
33/33 [==============================] - 0s 13ms/step - loss: 4.0950 - val_loss: 4.1550
Epoch 4/64
33/33 [==============================] - 1s 16ms/step - loss: 3.9717 - val_loss: 4.0980
Epoch 5/64
33/33 [==============================] - 1s 16ms/step - loss: 3.8473 - val_loss: 4.0413
Epoch 6/64
33/33 [==============================] - 0s 14ms/step - loss: 3.7223 - val_loss: 3.9855
Epoch 7/64
33/33 [==============================] - 0s 14ms/step - loss: 3.5977 - val_loss: 3.9310
Epoch 8/64
33/33 [==============================] - 0s 14ms/step - loss: 3.4740 - val_loss: 3.8780
Epoch 9/64
33/33 [==============================] - 0s 14ms/step - loss: 3.3517 - val_loss: 3.8266
Epoch 10/64
33/33 [==============================] - 0s 14ms/step - loss: 3.2316 - val_loss: 3.7774
Epoch 11/

In [301]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.446960210602205


In [302]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


### Model regularització
Hem afegit regularització per evitar overfitting del model. A més, com es quedava en loss 14, hem afegit també un callback per anar reduint
el learning rate en el moment en que la validation loss deixés de disminuir.

In [305]:
from tensorflow.keras import regularizers

In [310]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define a ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

In [314]:
#Provem el segon model
model = build_and_compile_model_reg(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.05, embeddings_regularizer=regularizers.l2(0.01))
# Entrenar el modelo
model.fit(train_dataset, epochs=80, validation_data=val_dataset,callbacks=[reduce_lr])

Epoch 1/80
33/33 [==============================] - 2s 21ms/step - loss: 6895.9434 - val_loss: 2587.4602 - lr: 0.0500
Epoch 2/80
33/33 [==============================] - 1s 17ms/step - loss: 1368.4388 - val_loss: 581.1382 - lr: 0.0500
Epoch 3/80
33/33 [==============================] - 1s 17ms/step - loss: 327.3352 - val_loss: 154.0800 - lr: 0.0500
Epoch 4/80
33/33 [==============================] - 1s 17ms/step - loss: 92.5883 - val_loss: 49.1179 - lr: 0.0500
Epoch 5/80
33/33 [==============================] - 1s 18ms/step - loss: 33.6034 - val_loss: 22.7459 - lr: 0.0500
Epoch 6/80
33/33 [==============================] - 1s 18ms/step - loss: 18.9829 - val_loss: 16.6858 - lr: 0.0500
Epoch 7/80
33/33 [==============================] - 1s 22ms/step - loss: 15.8766 - val_loss: 15.5408 - lr: 0.0500
Epoch 8/80
33/33 [==============================] - 1s 17ms/step - loss: 15.1505 - val_loss: 15.1971 - lr: 0.0500
Epoch 9/80
33/33 [==============================] - 1s 18ms/step - loss: 15.024

In [315]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 1ms/step
Correlación de Pearson: 0.30153116740473657


In [313]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


In [ ]:
modeloss.append("Word2vec 100MB Mean")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

## 500 MB amb Mean

In [319]:
#Convertir a vector amb model que tenguem guardat a la variable wv_model 
# Imprimir los pares de vectores y la puntuación de similitud asociada
wv_model = model500.wv
mapped = map_pairs(input_pairs, fixed_dictionary=diccionario, ) 
mapped_val = map_pairs(val, fixed_dictionary=diccionario, )
mapped_ts = map_pairs(ts, fixed_dictionary=diccionario, ) 

In [320]:

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped[0:])
x_val, y_val = pair_list_to_x_y(mapped_val[0:])
x_ts, y_ts = pair_list_to_x_y(mapped_ts[0:])

In [321]:

# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [322]:
pretrained_weights: Optional[np.ndarray] = None
if USE_PRETRAINED:
    if REMAP_EMBEDDINGS:
        pretrained_weights = np.zeros(
            (len(diccionario.token2id) + 1, wv_model.vector_size),  dtype=np.float32)
        for token, _id in diccionario.token2id.items():
            if token in wv_model:
                pretrained_weights[_id + 1] = wv_model[token]
            else:
                # In W2V, OOV will not have a representation. We will use 0.
                pass
    else:
        # Not recommended (this will consume A LOT of RAM)
        pretrained_weights = np.zeros((wv_model.vectors.shape[0] + 1, wv_model.vector_size,),  dtype=np.float32)
        pretrained_weights[1:, :] = wv_model.vectors

### Model 1

In [323]:
# Construir y compilar el modelo
#Hem augmentat el learning rate per a disminuir més ràpidament la loss i poder reduir el nombre d'epochs. Així faria overfitting
model = build_and_compile_model(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=32, validation_data=val_dataset)

Epoch 1/32
33/33 [==============================] - 2s 25ms/step - loss: 4.4601 - val_loss: 4.3262
Epoch 2/32
33/33 [==============================] - 1s 17ms/step - loss: 4.1395 - val_loss: 4.2135
Epoch 3/32
33/33 [==============================] - 1s 16ms/step - loss: 3.8045 - val_loss: 4.0985
Epoch 4/32
33/33 [==============================] - 1s 18ms/step - loss: 3.4485 - val_loss: 3.9843
Epoch 5/32
33/33 [==============================] - 1s 16ms/step - loss: 3.0842 - val_loss: 3.8738
Epoch 6/32
33/33 [==============================] - 1s 16ms/step - loss: 2.7246 - val_loss: 3.7690
Epoch 7/32
33/33 [==============================] - 1s 17ms/step - loss: 2.3797 - val_loss: 3.6711
Epoch 8/32
33/33 [==============================] - 1s 16ms/step - loss: 2.0565 - val_loss: 3.5813
Epoch 9/32
33/33 [==============================] - 1s 18ms/step - loss: 1.7597 - val_loss: 3.5000
Epoch 10/32
33/33 [==============================] - 1s 16ms/step - loss: 1.4922 - val_loss: 3.4272
Epoch 11/

In [324]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.44251699024533697


In [325]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


### Model 2

In [326]:
#Provem el segon model
model = build_and_compile_model_2(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=64, validation_data=val_dataset)

Epoch 1/64
33/33 [==============================] - 4s 30ms/step - loss: 2.2922 - val_loss: 2.0054
Epoch 2/64
33/33 [==============================] - 1s 19ms/step - loss: 1.7431 - val_loss: 1.4439
Epoch 3/64
33/33 [==============================] - 1s 18ms/step - loss: 1.2037 - val_loss: 0.9807
Epoch 4/64
33/33 [==============================] - 1s 21ms/step - loss: 0.8525 - val_loss: 0.7542
Epoch 5/64
33/33 [==============================] - 1s 23ms/step - loss: 0.6887 - val_loss: 0.6789
Epoch 6/64
33/33 [==============================] - 1s 20ms/step - loss: 0.6197 - val_loss: 0.6876
Epoch 7/64
33/33 [==============================] - 1s 24ms/step - loss: 0.5989 - val_loss: 0.6761
Epoch 8/64
33/33 [==============================] - 1s 22ms/step - loss: 0.5652 - val_loss: 0.6876
Epoch 9/64
33/33 [==============================] - 1s 20ms/step - loss: 0.5327 - val_loss: 0.6677
Epoch 10/64
33/33 [==============================] - 1s 20ms/step - loss: 0.5146 - val_loss: 0.6750
Epoch 11/

In [327]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.1926072257768498


In [328]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


### Model adamax

In [330]:
#Provem el segon model
model = build_and_compile_model_adamax(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=64, validation_data=val_dataset)

Epoch 1/64
33/33 [==============================] - 2s 23ms/step - loss: 4.4828 - val_loss: 4.3853
Epoch 2/64
33/33 [==============================] - 0s 15ms/step - loss: 4.3873 - val_loss: 4.3482
Epoch 3/64
33/33 [==============================] - 0s 15ms/step - loss: 4.3064 - val_loss: 4.3111
Epoch 4/64
33/33 [==============================] - 1s 17ms/step - loss: 4.2245 - val_loss: 4.2740
Epoch 5/64
33/33 [==============================] - 1s 16ms/step - loss: 4.1416 - val_loss: 4.2370
Epoch 6/64
33/33 [==============================] - 0s 14ms/step - loss: 4.0579 - val_loss: 4.2001
Epoch 7/64
33/33 [==============================] - 0s 15ms/step - loss: 3.9736 - val_loss: 4.1635
Epoch 8/64
33/33 [==============================] - 1s 21ms/step - loss: 3.8888 - val_loss: 4.1272
Epoch 9/64
33/33 [==============================] - 0s 13ms/step - loss: 3.8039 - val_loss: 4.0914
Epoch 10/64
33/33 [==============================] - 0s 13ms/step - loss: 3.7188 - val_loss: 4.0560
Epoch 11/

In [331]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.4374508794638199


In [332]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


### Model regularització
Hem afegit regularització per evitar overfitting del model. A més, com es quedava en loss 14, hem afegit també un callback per anar reduint
el learning rate en el moment en que la validation loss deixés de disminuir.

In [333]:
from tensorflow.keras import regularizers

In [334]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define a ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

In [335]:
#Provem el segon model
model = build_and_compile_model_reg(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.05, embeddings_regularizer=regularizers.l2(0.01))
# Entrenar el modelo
model.fit(train_dataset, epochs=80, validation_data=val_dataset,callbacks=[reduce_lr])

Epoch 1/80
33/33 [==============================] - 2s 31ms/step - loss: 18725.4375 - val_loss: 8968.8271 - lr: 0.0500
Epoch 2/80
33/33 [==============================] - 1s 19ms/step - loss: 5339.5483 - val_loss: 2730.5757 - lr: 0.0500
Epoch 3/80
33/33 [==============================] - 1s 16ms/step - loss: 1691.2509 - val_loss: 914.8701 - lr: 0.0500
Epoch 4/80
33/33 [==============================] - 1s 16ms/step - loss: 582.8638 - val_loss: 328.0367 - lr: 0.0500
Epoch 5/80
33/33 [==============================] - 1s 17ms/step - loss: 214.1270 - val_loss: 125.2687 - lr: 0.0500
Epoch 6/80
33/33 [==============================] - 1s 17ms/step - loss: 84.5177 - val_loss: 52.6053 - lr: 0.0500
Epoch 7/80
33/33 [==============================] - 1s 18ms/step - loss: 37.9330 - val_loss: 26.8575 - lr: 0.0500
Epoch 8/80
33/33 [==============================] - 1s 19ms/step - loss: 21.5930 - val_loss: 17.8833 - lr: 0.0500
Epoch 9/80
33/33 [==============================] - 1s 16ms/step - loss:

In [336]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.3061931245836725


In [337]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


## Fasttext

In [ ]:
#Convertir a vector amb model que tenguem guardat a la variable wv_model 
# Imprimir los pares de vectores y la puntuación de similitud asociada
wv_model = ft_model.wv
mapped = map_pairs(input_pairs, fixed_dictionary=diccionario, ) 
mapped_val = map_pairs(val, fixed_dictionary=diccionario, )
mapped_ts = map_pairs(ts, fixed_dictionary=diccionario, ) 

In [ ]:

# Obtener las listas de train y test
x_train, y_train = pair_list_to_x_y(mapped[0:])
x_val, y_val = pair_list_to_x_y(mapped_val[0:])
x_ts, y_ts = pair_list_to_x_y(mapped_ts[0:])

In [ ]:

# Preparar los conjuntos de datos de entrenamiento y validación
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=len(x_train)).batch(batch_size)
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.batch(batch_size)

In [ ]:
pretrained_weights: Optional[np.ndarray] = None
if USE_PRETRAINED:
    if REMAP_EMBEDDINGS:
        pretrained_weights = np.zeros(
            (len(diccionario.token2id) + 1, wv_model.vector_size),  dtype=np.float32)
        for token, _id in diccionario.token2id.items():
            if token in wv_model:
                pretrained_weights[_id + 1] = wv_model[token]
            else:
                # In W2V, OOV will not have a representation. We will use 0.
                pass
    else:
        # Not recommended (this will consume A LOT of RAM)
        pretrained_weights = np.zeros((wv_model.vectors.shape[0] + 1, wv_model.vector_size,),  dtype=np.float32)
        pretrained_weights[1:, :] = wv_model.vectors

### Model 1

In [ ]:
# Construir y compilar el modelo
#Hem augmentat el learning rate per a disminuir més ràpidament la loss i poder reduir el nombre d'epochs. Així faria overfitting
model = build_and_compile_model(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=32, validation_data=val_dataset)

Epoch 1/32
33/33 [==============================] - 2s 19ms/step - loss: 4.3234 - val_loss: 4.1764
Epoch 2/32
33/33 [==============================] - 0s 15ms/step - loss: 3.8426 - val_loss: 4.0016
Epoch 3/32
33/33 [==============================] - 1s 15ms/step - loss: 3.3471 - val_loss: 3.8308
Epoch 4/32
33/33 [==============================] - 1s 16ms/step - loss: 2.8443 - val_loss: 3.6738
Epoch 5/32
33/33 [==============================] - 1s 16ms/step - loss: 2.3673 - val_loss: 3.5362
Epoch 6/32
33/33 [==============================] - 1s 16ms/step - loss: 1.9390 - val_loss: 3.4183
Epoch 7/32
33/33 [==============================] - 1s 17ms/step - loss: 1.5673 - val_loss: 3.3177
Epoch 8/32
33/33 [==============================] - 1s 20ms/step - loss: 1.2514 - val_loss: 3.2330
Epoch 9/32
33/33 [==============================] - 1s 16ms/step - loss: 0.9923 - val_loss: 3.1626
Epoch 10/32
33/33 [==============================] - 1s 16ms/step - loss: 0.7851 - val_loss: 3.1053
Epoch 11/

In [ ]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.4422017787406168


In [ ]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


### Model 2

In [ ]:
#Provem el segon model
model = build_and_compile_model_2(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=64, validation_data=val_dataset)

Epoch 1/64
33/33 [==============================] - 3s 32ms/step - loss: 2.2909 - val_loss: 2.0126
Epoch 2/64
33/33 [==============================] - 1s 22ms/step - loss: 1.7397 - val_loss: 1.4488
Epoch 3/64
33/33 [==============================] - 1s 22ms/step - loss: 1.1643 - val_loss: 0.9326
Epoch 4/64
33/33 [==============================] - 1s 19ms/step - loss: 0.7795 - val_loss: 0.6988
Epoch 5/64
33/33 [==============================] - 1s 18ms/step - loss: 0.6393 - val_loss: 0.6422
Epoch 6/64
33/33 [==============================] - 1s 18ms/step - loss: 0.5649 - val_loss: 0.6322
Epoch 7/64
33/33 [==============================] - 1s 22ms/step - loss: 0.5050 - val_loss: 0.6161
Epoch 8/64
33/33 [==============================] - 1s 21ms/step - loss: 0.4649 - val_loss: 0.6253
Epoch 9/64
33/33 [==============================] - 1s 24ms/step - loss: 0.4218 - val_loss: 0.6252
Epoch 10/64
33/33 [==============================] - 1s 26ms/step - loss: 0.3820 - val_loss: 0.6220
Epoch 11/

In [ ]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.24598748743437876


In [ ]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


In [ ]:
modeloss.append("Word2vec 100MB Mean")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

### Model adamax

In [ ]:
#Provem el segon model
model = build_and_compile_model_adamax(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=64, validation_data=val_dataset)

Epoch 1/64
33/33 [==============================] - 2s 18ms/step - loss: 4.3603 - val_loss: 4.2699
Epoch 2/64
33/33 [==============================] - 0s 13ms/step - loss: 4.2171 - val_loss: 4.2123
Epoch 3/64
33/33 [==============================] - 0s 13ms/step - loss: 4.0950 - val_loss: 4.1550
Epoch 4/64
33/33 [==============================] - 1s 16ms/step - loss: 3.9717 - val_loss: 4.0980
Epoch 5/64
33/33 [==============================] - 1s 16ms/step - loss: 3.8473 - val_loss: 4.0413
Epoch 6/64
33/33 [==============================] - 0s 14ms/step - loss: 3.7223 - val_loss: 3.9855
Epoch 7/64
33/33 [==============================] - 0s 14ms/step - loss: 3.5977 - val_loss: 3.9310
Epoch 8/64
33/33 [==============================] - 0s 14ms/step - loss: 3.4740 - val_loss: 3.8780
Epoch 9/64
33/33 [==============================] - 0s 14ms/step - loss: 3.3517 - val_loss: 3.8266
Epoch 10/64
33/33 [==============================] - 0s 14ms/step - loss: 3.2316 - val_loss: 3.7774
Epoch 11/

In [ ]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.446960210602205


In [ ]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


### Model regularització
Hem afegit regularització per evitar overfitting del model. A més, com es quedava en loss 14, hem afegit també un callback per anar reduint
el learning rate en el moment en que la validation loss deixés de disminuir.

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define a ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

In [ ]:
#Provem el segon model
model = build_and_compile_model_reg(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.05, embeddings_regularizer=regularizers.l2(0.01))
# Entrenar el modelo
model.fit(train_dataset, epochs=80, validation_data=val_dataset,callbacks=[reduce_lr])

Epoch 1/80
33/33 [==============================] - 2s 21ms/step - loss: 6895.9434 - val_loss: 2587.4602 - lr: 0.0500
Epoch 2/80
33/33 [==============================] - 1s 17ms/step - loss: 1368.4388 - val_loss: 581.1382 - lr: 0.0500
Epoch 3/80
33/33 [==============================] - 1s 17ms/step - loss: 327.3352 - val_loss: 154.0800 - lr: 0.0500
Epoch 4/80
33/33 [==============================] - 1s 17ms/step - loss: 92.5883 - val_loss: 49.1179 - lr: 0.0500
Epoch 5/80
33/33 [==============================] - 1s 18ms/step - loss: 33.6034 - val_loss: 22.7459 - lr: 0.0500
Epoch 6/80
33/33 [==============================] - 1s 18ms/step - loss: 18.9829 - val_loss: 16.6858 - lr: 0.0500
Epoch 7/80
33/33 [==============================] - 1s 22ms/step - loss: 15.8766 - val_loss: 15.5408 - lr: 0.0500
Epoch 8/80
33/33 [==============================] - 1s 17ms/step - loss: 15.1505 - val_loss: 15.1971 - lr: 0.0500
Epoch 9/80
33/33 [==============================] - 1s 18ms/step - loss: 15.024

In [ ]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 1ms/step
Correlación de Pearson: 0.30153116740473657


In [ ]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


## Random embedings

In [354]:
USE_PRETRAINED = False

In [352]:
#Creem un diccionari paraula: vector aleatori uniforme
dicc = diccionario.token2id.copy()
for par in dicc:
    dicc[par] = np.random.uniform(low=0.0, high=1.0, size=100)

In [357]:
pretrained_weights: Optional[np.ndarray] = None
if USE_PRETRAINED:
    if REMAP_EMBEDDINGS:
        pretrained_weights = np.zeros(
            (len(diccionario.token2id) + 1, wv_model.vector_size),  dtype=np.float32)
        for token, _id in diccionario.token2id.items():
            if token in wv_model:
                pretrained_weights[_id + 1] = wv_model[token]
            else:
                # In W2V, OOV will not have a representation. We will use 0.
                pass
    else:
        # Not recommended (this will consume A LOT of RAM)
        pretrained_weights = np.zeros((wv_model.vectors.shape[0] + 1, wv_model.vector_size,),  dtype=np.float32)
        pretrained_weights[1:, :] = wv_model.vectors
else:
    if REMAP_EMBEDDINGS:
        pretrained_weights = np.zeros(
            (len(diccionario.token2id) + 1, wv_model.vector_size),  dtype=np.float32)
        for token, _id in diccionario.token2id.items():
            pretrained_weights[_id + 1] = dicc[token]


### Model 1

In [358]:
# Construir y compilar el modelo
#Hem augmentat el learning rate per a disminuir més ràpidament la loss i poder reduir el nombre d'epochs. Així faria overfitting
model = build_and_compile_model(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=32, validation_data=val_dataset)

Epoch 1/32
33/33 [==============================] - 2s 25ms/step - loss: 6.4000 - val_loss: 6.2805
Epoch 2/32
33/33 [==============================] - 1s 17ms/step - loss: 6.0190 - val_loss: 6.0170
Epoch 3/32
33/33 [==============================] - 1s 16ms/step - loss: 5.0817 - val_loss: 5.3120
Epoch 4/32
33/33 [==============================] - 1s 16ms/step - loss: 2.9226 - val_loss: 3.7226
Epoch 5/32
33/33 [==============================] - 1s 17ms/step - loss: 1.0159 - val_loss: 3.0167
Epoch 6/32
33/33 [==============================] - 1s 16ms/step - loss: 0.4133 - val_loss: 2.9930
Epoch 7/32
33/33 [==============================] - 1s 16ms/step - loss: 0.2057 - val_loss: 2.9626
Epoch 8/32
33/33 [==============================] - 1s 16ms/step - loss: 0.1302 - val_loss: 2.9637
Epoch 9/32
33/33 [==============================] - 1s 16ms/step - loss: 0.0989 - val_loss: 2.9618
Epoch 10/32
33/33 [==============================] - 1s 16ms/step - loss: 0.0861 - val_loss: 2.9674
Epoch 11/

In [359]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.3426584368705931


In [360]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


### Model 2

In [361]:
#Provem el segon model
model = build_and_compile_model_2(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=64, validation_data=val_dataset)

Epoch 1/64
33/33 [==============================] - 3s 26ms/step - loss: 2.2126 - val_loss: 1.8421
Epoch 2/64
33/33 [==============================] - 1s 20ms/step - loss: 1.4758 - val_loss: 1.1426
Epoch 3/64
33/33 [==============================] - 1s 22ms/step - loss: 0.9126 - val_loss: 0.7773
Epoch 4/64
33/33 [==============================] - 1s 21ms/step - loss: 0.7072 - val_loss: 0.6737
Epoch 5/64
33/33 [==============================] - 1s 20ms/step - loss: 0.6466 - val_loss: 0.6486
Epoch 6/64
33/33 [==============================] - 1s 20ms/step - loss: 0.6301 - val_loss: 0.6415
Epoch 7/64
33/33 [==============================] - 1s 20ms/step - loss: 0.6057 - val_loss: 0.6417
Epoch 8/64
33/33 [==============================] - 1s 22ms/step - loss: 0.5859 - val_loss: 0.6379
Epoch 9/64
33/33 [==============================] - 1s 22ms/step - loss: 0.5595 - val_loss: 0.6436
Epoch 10/64
33/33 [==============================] - 1s 22ms/step - loss: 0.5489 - val_loss: 0.6475
Epoch 11/

In [ ]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.24598748743437876


In [ ]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


In [ ]:
modeloss.append("Word2vec 100MB Mean")
P_Embeding.append(correlationE)
P_Model.append(correlationM)

### Model adamax

In [ ]:
#Provem el segon model
model = build_and_compile_model_adamax(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.01, )
# Entrenar el modelo
model.fit(train_dataset, epochs=64, validation_data=val_dataset)

Epoch 1/64
33/33 [==============================] - 2s 18ms/step - loss: 4.3603 - val_loss: 4.2699
Epoch 2/64
33/33 [==============================] - 0s 13ms/step - loss: 4.2171 - val_loss: 4.2123
Epoch 3/64
33/33 [==============================] - 0s 13ms/step - loss: 4.0950 - val_loss: 4.1550
Epoch 4/64
33/33 [==============================] - 1s 16ms/step - loss: 3.9717 - val_loss: 4.0980
Epoch 5/64
33/33 [==============================] - 1s 16ms/step - loss: 3.8473 - val_loss: 4.0413
Epoch 6/64
33/33 [==============================] - 0s 14ms/step - loss: 3.7223 - val_loss: 3.9855
Epoch 7/64
33/33 [==============================] - 0s 14ms/step - loss: 3.5977 - val_loss: 3.9310
Epoch 8/64
33/33 [==============================] - 0s 14ms/step - loss: 3.4740 - val_loss: 3.8780
Epoch 9/64
33/33 [==============================] - 0s 14ms/step - loss: 3.3517 - val_loss: 3.8266
Epoch 10/64
33/33 [==============================] - 0s 14ms/step - loss: 3.2316 - val_loss: 3.7774
Epoch 11/

In [ ]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 2ms/step
Correlación de Pearson: 0.446960210602205


In [ ]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


### Model regularització
Hem afegit regularització per evitar overfitting del model. A més, com es quedava en loss 14, hem afegit també un callback per anar reduint
el learning rate en el moment en que la validation loss deixés de disminuir.

In [ ]:
from tensorflow.keras import regularizers

In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define a ReduceLROnPlateau callback
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.001)

In [ ]:
#Provem el segon model
model = build_and_compile_model_reg(pretrained_weights=pretrained_weights, trainable=True, learning_rate=0.05, embeddings_regularizer=regularizers.l2(0.01))
# Entrenar el modelo
model.fit(train_dataset, epochs=80, validation_data=val_dataset,callbacks=[reduce_lr])

Epoch 1/80
33/33 [==============================] - 2s 21ms/step - loss: 6895.9434 - val_loss: 2587.4602 - lr: 0.0500
Epoch 2/80
33/33 [==============================] - 1s 17ms/step - loss: 1368.4388 - val_loss: 581.1382 - lr: 0.0500
Epoch 3/80
33/33 [==============================] - 1s 17ms/step - loss: 327.3352 - val_loss: 154.0800 - lr: 0.0500
Epoch 4/80
33/33 [==============================] - 1s 17ms/step - loss: 92.5883 - val_loss: 49.1179 - lr: 0.0500
Epoch 5/80
33/33 [==============================] - 1s 18ms/step - loss: 33.6034 - val_loss: 22.7459 - lr: 0.0500
Epoch 6/80
33/33 [==============================] - 1s 18ms/step - loss: 18.9829 - val_loss: 16.6858 - lr: 0.0500
Epoch 7/80
33/33 [==============================] - 1s 22ms/step - loss: 15.8766 - val_loss: 15.5408 - lr: 0.0500
Epoch 8/80
33/33 [==============================] - 1s 17ms/step - loss: 15.1505 - val_loss: 15.1971 - lr: 0.0500
Epoch 9/80
33/33 [==============================] - 1s 18ms/step - loss: 15.024

In [ ]:
y_pred = model.predict(x_ts)
#print(y_pred)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationM, _ = pearsonr(y_pred.flatten(), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationM}")

16/16 [==============================] - 0s 1ms/step
Correlación de Pearson: 0.30153116740473657


In [ ]:
# Podemos evaluar el modelo si sólo utilizamos COS similarity. (Depende completamente de los Word Embeddings)
y_pred_baseline = []

vs1, vs2 = x_ts
for v1,v2 in zip(vs1,vs2):
    d= 1.0 - spatial.distance.cosine(v1, v2)
    y_pred_baseline.append(d)
# Calcular la correlación de Pearson entre las predicciones y los datos de prueba
correlationE, _ = pearsonr(np.array(y_pred_baseline), y_ts.flatten())
# Imprimir el coeficiente de correlación de Pearson
print(f"Correlación de Pearson: {correlationE}") 

Correlación de Pearson: 0.23536080252591637


In [ ]:
modeloss.append("Word2vec 100MB Mean")
P_Embeding.append(correlationE)
P_Model.append(correlationM)